### Carregamento

In [1]:
import pandas as pd
# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV com separador ';'
    dados = pd.read_csv(caminho_arquivo, sep=',')
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados
# Caminho
caminho_arquivo = '../data/raw/uber_stock_data.csv'
# Carrega os dados com o separador correto
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)


,Date,Adj Close,Close,High,Low,Open,Volume
0,2019-05-10,41.570000,41.570000,45.000000,41.060001,42.000000,186322500
1,2019-05-13,37.099998,37.099998,39.240002,36.080002,38.790001,79442400
2,2019-05-14,39.959999,39.959999,39.959999,36.849998,38.310001,46661100
3,2019-05-15,41.290001,41.290001,41.880001,38.950001,39.369999,36086100
4,2019-05-16,43.000000,43.000000,44.060001,41.250000,41.480000,38115500
5,2019-05-17,41.910000,41.910000,43.290001,41.270000,41.980000,20225700
6,2019-05-20,41.590000,41.590000,41.680000,39.459999,41.189999,29222300
7,2019-05-21,41.500000,41.500000,42.240002,41.250000,42.000000,10802900
8,2019-05-22,41.250000,41.250000,41.279999,40.500000,41.049999,9089500
9,2019-05-23,40.470001,40.470001,41.090000,40.020000,40.799999,11119900


### Tamanho da base

In [2]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 1444, 'Número de Colunas': 7}

### Verificação nulos

In [3]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [4]:
def verificar_valores_duplicados(df, colunas=None):
    """
    Verifica valores duplicados em um DataFrame, fornecendo análise detalhada.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado
    - colunas (list, opcional): Lista específica de colunas para verificar duplicatas.
                               Se None, verifica todas as colunas.
    
    Retorna:
    - dict: Dicionário com informações sobre duplicatas para cada coluna analisada
    """
    resultado = {}
    
    # Se nenhuma coluna específica for fornecida, usar todas as colunas
    if colunas is None:
        colunas = df.columns
    
    # Análise de duplicatas por coluna
    for coluna in colunas:
        # Encontrar valores duplicados
        duplicados = df[df[coluna].duplicated(keep='first')]
        qtd_duplicados = len(duplicados)
        
        if qtd_duplicados > 0:
            # Contagem de ocorrências de cada valor duplicado
            valores_duplicados = df[coluna].value_counts()[df[coluna].value_counts() > 1]
            
            resultado[coluna] = {
                'possui_duplicados': True,
                'quantidade_duplicados': qtd_duplicados,
                'valores_e_contagem': valores_duplicados.to_dict()
            }
        else:
            resultado[coluna] = {
                'possui_duplicados': False,
                'quantidade_duplicados': 0,
                'valores_e_contagem': {}
            }
    
    # Exibir resumo
    print("=== Análise de Valores Duplicados ===\n")
    
    for coluna, info in resultado.items():
        print(f"\nColuna: {coluna}")
        if info['possui_duplicados']:
            print(f"Status: Possui {info['quantidade_duplicados']} valores duplicados")
            print("Valores duplicados e suas ocorrências:")
            for valor, contagem in info['valores_e_contagem'].items():
                print(f"  - Valor: {valor} | Ocorrências: {contagem}")
        else:
            print("Status: Não possui valores duplicados")
    
    return resultado

# Executar verificação de valores duplicados
resultados_duplicados = verificar_valores_duplicados(dados)

=== Análise de Valores Duplicados ===


Coluna: Date
Status: Não possui valores duplicados

Coluna: Adj Close
Status: Possui 217 valores duplicados
Valores duplicados e suas ocorrências:
  - Valor: 33.04999924 | Ocorrências: 4
  - Valor: 30.68000031 | Ocorrências: 4
  - Valor: 45.72000122 | Ocorrências: 3
  - Valor: 39.70000076 | Ocorrências: 3
  - Valor: 31.19000053 | Ocorrências: 3
  - Valor: 31.12000084 | Ocorrências: 3
  - Valor: 43.09000015 | Ocorrências: 3
  - Valor: 30.93000031 | Ocorrências: 3
  - Valor: 27.81999969 | Ocorrências: 3
  - Valor: 40.61999893 | Ocorrências: 3
  - Valor: 73.25 | Ocorrências: 3
  - Valor: 31.48999977 | Ocorrências: 3
  - Valor: 33.40999985 | Ocorrências: 3
  - Valor: 44.52000046 | Ocorrências: 3
  - Valor: 44.36000061 | Ocorrências: 3
  - Valor: 30.82999992 | Ocorrências: 3
  - Valor: 31.68000031 | Ocorrências: 3
  - Valor: 34.25999832 | Ocorrências: 3
  - Valor: 33.40000153 | Ocorrências: 3
  - Valor: 31.18000031 | Ocorrências: 3
  - Valor: 31.5 | O

### Verificação dados númericos

In [6]:
def verificar_colunas_numericas(df, colunas):
    """
    Verifica se as colunas especificadas contêm apenas valores numéricos.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    - colunas (list): Lista de nomes das colunas a serem verificadas.

    Retorna:
    - dict: Dicionário com o nome da coluna e um booleano indicando se contém apenas números.
    - pandas.DataFrame: DataFrame com as inconsistências encontradas.
    """
    resultado = {}
    inconsistencias = []

    for coluna in colunas:
        if coluna in df.columns:
            # Verifica se todos os valores na coluna são numéricos
            col_numerica = pd.to_numeric(df[coluna], errors='coerce')
            resultado[coluna] = col_numerica.notnull().all()
            if not resultado[coluna]:
                # Adiciona as inconsistências à lista
                posicoes_inconsistentes = col_numerica[col_numerica.isnull()].index.tolist()
                for posicao in posicoes_inconsistentes:
                    inconsistencias.append({'Coluna': coluna, 'Posição': posicao, 'Valor': df.at[posicao, coluna]})
        else:
            resultado[coluna] = False
            print(f"⚠️ Coluna '{coluna}' não encontrada no DataFrame.")

    # Cria um DataFrame com as inconsistências
    df_inconsistencias = pd.DataFrame(inconsistencias)

    return resultado, df_inconsistencias

# Colunas a serem verificadas
colunas_a_verificar = [
   'Date', 'Adj Close', 'Close',
   'High', 'Low', 'Open',
   'Volume'
]

# Verificar as colunas no DataFrame 'dados'
resultado_verificacao, df_inconsistencias = verificar_colunas_numericas(dados, colunas_a_verificar)

# Exibir o resultado da verificação
for coluna, is_numerica in resultado_verificacao.items():
    status = "contém apenas números" if is_numerica else "não contém apenas números"
    print(f"Coluna '{coluna}': {status}")

# Exibir as inconsistências, se houver
if not df_inconsistencias.empty:
    print("\nInconsistências encontradas:")
    display(df_inconsistencias)
else:
    print("\nNenhuma inconsistência encontrada nas colunas verificadas.")

Coluna 'Date': não contém apenas números
Coluna 'Adj Close': contém apenas números
Coluna 'Close': contém apenas números
Coluna 'High': contém apenas números
Coluna 'Low': contém apenas números
Coluna 'Open': contém apenas números
Coluna 'Volume': contém apenas números

Inconsistências encontradas:


,Coluna,Posição,Valor
0,Date,0,2019-05-10
1,Date,1,2019-05-13
2,Date,2,2019-05-14
3,Date,3,2019-05-15
4,Date,4,2019-05-16
...,...,...,...
1439,Date,1439,2025-01-30
1440,Date,1440,2025-01-31
1441,Date,1441,2025-02-03
1442,Date,1442,2025-02-04


### Verificação tipos dos dados

In [5]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,Date,object
1,Adj Close,float64
2,Close,float64
3,High,float64
4,Low,float64
5,Open,float64
6,Volume,int64


### Valores máx/min

In [7]:
def valores_max_min(df):
    """
    Percorre todas as colunas numéricas (int ou float) de um DataFrame e retorna uma tabela com os valores máximos e mínimos de cada coluna.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores máximos e mínimos de cada coluna numérica.
    """
    # Selecionar apenas colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Criar listas para armazenar os resultados
    colunas = []
    valores_min = []
    valores_max = []
    
    # Percorrer cada coluna numérica e calcular os valores máximo e mínimo
    for coluna in colunas_numericas:
        colunas.append(coluna)
        valores_min.append(df[coluna].min())
        valores_max.append(df[coluna].max())
    
    # Criar o DataFrame final com todos os resultados de uma vez
    tabela_max_min = pd.DataFrame({
        'Coluna': colunas,
        'Valor Mínimo': valores_min,
        'Valor Máximo': valores_max
    })
    
    return tabela_max_min

# Aplicar a função e exibir a tabela de valores máximos e mínimos
tabela_max_min = valores_max_min(dados)
tabela_max_min

,Coluna,Valor Mínimo,Valor Máximo
0,Adj Close,1.482000e+01,8.634000e+01
1,Close,1.482000e+01,8.634000e+01
2,High,1.780000e+01,8.700000e+01
3,Low,1.371000e+01,8.418000e+01
4,Open,1.596000e+01,8.564000e+01
5,Volume,3.380000e+06,3.642318e+08


### Conversão/otimização

In [8]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'Date': 'datetime64[ns]'
}
 
dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'Date' convertida para 'datetime64[ns]' com sucesso.


Date         datetime64[ns]
Adj Close           float64
Close               float64
High                float64
Low                 float64
Open                float64
Volume                int64
dtype: object

### Carga

In [9]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/uber_stock_data_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/uber_stock_data_atualizado.csv'
